In [332]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from scipy.stats import skew
import seaborn as sns
import sklearn
import warnings
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.linear_model import Lasso


In [334]:
train_clean = pd.read_csv('train_clean.csv', dtype={'MSSubClass': str})  
train = pd.read_csv('train.csv', dtype={'MSSubClass': str})

Remove outlier records we previously identified as being harmful to the model

In [335]:
train_clean = train_clean[(train_clean.Id != 1299) & (train_clean.Id != 524)]

In [336]:
div = len(train_clean) // 5
train_clean['fold'] = 999
train_clean['m1'] = 0
train_clean['m2'] = 0
train_shuf = train_clean.sample(frac=1).copy()


for i in range(1, 6):
    start = div*i - div
    end = div*i
    train_shuf.iloc[start:end,:]['fold']= i
train_shuf.iloc[end:]['fold'] = 5    

/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [337]:
print(train_shuf.fold.head())
train_shuf.fold.tail()

955     1
1089    1
236     1
1226    1
220     1
Name: fold, dtype: int64


901     5
2       5
778     5
1012    5
1452    5
Name: fold, dtype: int64

In [338]:
test_folds = []
for i in train_shuf.fold.unique():
    test = train_shuf[train_shuf.fold == i]
    tr = train_clean[train_shuf.fold != i]
    X_tr =  tr.drop(['SalePrice', 'Id', 'fold'], axis=1)
    X_test =  test.drop(['SalePrice', 'Id', 'fold'], axis=1)
    y_tr = tr['SalePrice']    
    
    ls = Lasso(alpha=0.001)
    ls.fit(X_tr, y_tr)
    test['m1'] = ls.predict(X_test)
    
    params = {'eta':0.01, 'max_depth':4, 'min_child_weight':4, 'subsample':0.4, 'colsample_bytree':0.8, 'scale_pos_weight':1}
    dtr = xgb.DMatrix(X_tr, y_tr)
    dtest = xgb.DMatrix(X_test)
    num_rounds = 1334
    bst = xgb.train(params, dtr, num_rounds)
    test['m2'] = bst.predict(dtest)
    
    test_folds.append(test)

/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [339]:
tr_meta = pd.concat(test_folds)

In [340]:
test_set = pd.read_csv('test_clean.csv', dtype={'MSSubClass': str})
ids = test_set.Id.values
test_set = test_set.drop('Id', axis=1)


In [341]:
X_tr =  train_clean.drop(['SalePrice', 'm1', 'm2', 'fold', 'Id'], axis=1)
y_tr = train_clean['SalePrice']    
ls = Lasso(alpha=0.001)
ls.fit(X_tr, y_tr)


Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [342]:
params = {'eta':0.01, 'max_depth':4, 'min_child_weight':4, 'subsample':0.4, 'colsample_bytree':0.8, 'scale_pos_weight':1}
dtr = xgb.DMatrix(X_tr, y_tr)
dtest = xgb.DMatrix(test_set)
num_rounds = 1334
bst = xgb.train(params, dtr, num_rounds)
test_set['m1'] = ls.predict(test_set)
test_set['m2'] = bst.predict(dtest)


In [343]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

In [393]:
poly = PolynomialFeatures(2)
tr_meta_expand = tr_meta[['m1', 'm2', 'OverallQual', 'Neighborhood_Crawfor', 'GrLivArea']]
tr_meta_expand = poly.fit_transform(tr_meta_expand)

In [394]:
expanded = pd.DataFrame(tr_meta_expand, index=tr_meta.index)

In [395]:
tr_meta_expand= tr_meta.merge(expanded, left_index=True, right_index=True)

In [396]:
scores = []
for i in tr_meta.fold.unique():
    test = tr_meta_expand[train_shuf.fold == i]
    tr = tr_meta_expand[tr_meta.fold != i]
    tr_cols = list(expanded.columns)
    X_tr =  tr[tr_cols]
    X_test =  test[tr_cols]
    y_test = test['SalePrice']
    y_tr = tr['SalePrice']
    
    r = Ridge(alpha=1)
    r.fit(X_tr, y_tr)
    preds = r.predict(X_test)
    score = np.sqrt(np.mean((y_test.values - preds) **2))
    print(score)
    scores.append(score)
print('mean score: {}'.format(np.mean(scores)))

0.11824828229
0.110746159832
0.113699240715
0.105782327504
0.106607056683
mean score: 0.11101661340465008


In [398]:
X_tr =  tr_meta_expand[tr_cols]
y_tr = tr_meta_expand['SalePrice']    


X_test = poly.transform(test_set[['m1', 'm2', 'OverallQual', 'Neighborhood_Crawfor', 'GrLivArea']])

#X_test =  test_set[['m1', 'm2', 'OverallQual']]



r = Ridge(alpha=1)
r.fit(X_tr, y_tr)


Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [399]:
preds = np.expm1(r.predict(X_test))

In [401]:
preds

array([ 123739.30556718,  154044.41918296,  182863.54366044, ...,
        167163.63741078,  118725.55382915,  227207.69135958])

In [400]:
preds.shape

(1459,)

In [275]:
ids.shape

(1459,)

In [402]:
solution = pd.DataFrame({"id":ids, "SalePrice":preds}, columns=['id', 'SalePrice'])

solution.to_csv("stack_poly.csv", index = False)

This one scores 0.11945

Let's save the model so we can use it later

In [115]:
from sklearn.externals import joblib
joblib.dump(bestLassoEst, 'poly_features_35_2.pkl')

['poly_features_35_2.pkl']